In [ ]:
import pandas as pd
import numpy as np
"""graficas"""
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
"""preprocesar"""
from sklearn.preprocessing import StandardScaler, label_binarize
"""dividir datos"""
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
"""validar modelo"""
from sklearn.model_selection import validation_curve, StratifiedKFold, GridSearchCV, cross_validate
"""metricas de evaluacion"""
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, roc_curve, auc, make_scorer
"""modelos ML"""
from sklearn.ensemble import RandomForestClassifier


""" crear Funcion para matriz de confusion """
def crearMatriz_confusion(y_pred,y_test):
    cm= confusion_matrix(y_pred, y_test)
    plt.figure(figsize = (8,6))

    values = ["{0:0.0f}".format(x) for x in cm.flatten()]

    """ calcular y obtener valores"""
    percentages = ["{0:.2%}".format(x) for x in cm.flatten()/np.sum(cm)]
    combined = ["{}\n{}\n".format(j, k) for j, k in zip(values, percentages)]
    combined = np.asarray(combined).reshape(4,4)

    """crear matriz"""
    b = sns.heatmap(cm, annot=combined, fmt="")
    b.set_xticklabels(['DDoS(0)','Dos(1)','Normal(2)','Probe(3)'],fontsize=10)
    b.set_yticklabels(['DDoS(0)','Dos(1)','Normal(2)','Probe(3)'],fontsize=10)
    """etiquetas"""
    b.set(title='Confusion Matrix')
    b.set(xlabel='Prediccion', ylabel='Actual',)
    #plt.savefig('matrizconfusion_mobiln.png')
    plt.show()

"""crear funcion para encontrar metricas del modelo """
def encontrar_metricas(y_test,y_pred):
    """ encontrar la precision del general del modelo"""
    accuracy = accuracy_score(y_test, y_pred)*100
    precision_ma = precision_score(y_test, y_pred, average='weighted')*100
    recall_ma = recall_score(y_test, y_pred, average='weighted')*100
    f1score_ma = f1_score(y_test, y_pred, average='weighted')*100
    return print('Accuracy: %s \nPrecision: %s \nRecall: %s \nF1-Score: %s' %(accuracy, precision_ma, recall_ma, f1score_ma))


df_traffic = pd.read_csv('../3_Scaling/DB_34/balance_total4clases_34200.csv', low_memory=False)
"""matriz de caracteristicas"""
caracteristicas = ['Dur','protoTcp','protoUdp','portSystem','portUser','portDynamic','TotPkts', 'SrcPkts', 'DstPkts', 
                    'TotBytes', 'SrcBytes', 'DstBytes']
print("############# 1. Caracteristicas: %s ###############" %len(caracteristicas) )

"""seleccion de variables"""
X= df_traffic[caracteristicas].values
y=df_traffic.iloc[:,-1].values

print("############# 2. Division de datos #############")
"""division de datos"""
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index,"\n")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

"""estandarizar datos"""
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

"""validacion cruzada"""
skf = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

print("############ 3. Seleccionando modelo #############")
parametros_grid = [{'n_estimators': [100, 200, 300, 400, 500, 1000], 'criterion' :['gini', 'entropy','log_loss'],
                    'max_depth' : [5, 6, 7, 8, 9, 10, 20, 30],
                    'max_features': ['auto', 'sqrt', 'log2'],
                    'bootstrap': [True, False],
                    'class_weight':['balanced', 'balanced_subsample'],
                    'ccp_alpha':[0.001, 0.01,0.5, 1, 5, 10],
                }]

rf = RandomForestClassifier(metric = "minkowski",  p = 2)
grid_search= GridSearchCV(estimator = rf, param_grid = parametros_grid, cv = skf, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Mejores hiperparametros: ", grid_search.best_params_)
print("Mejor puntuacion: ", grid_search.best_score_)
print("Modelo: ", grid_search.best_estimator_)

print("############ 4. Entrenando modelo #############")
clf_rf = RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'],
                                criterion = grid_search.best_params_['criterion'],
                                max_depth = grid_search.best_params_['max_depth'],
                                max_features = grid_search.best_params_['max_features'],
                                bootstrap = grid_search.best_params_['bootstrap'],
                                class_weight = grid_search.best_params_['class_weight'],
                                ccp_alpha = grid_search.best_params_['ccp_alpha'])
clf_rf = clf_rf.fit(X_train,y_train)
y_pred = clf_rf.predict(X_test)


print("############ 5. Metricas del modelo #############")
""" metricas"""
encontrar_metricas(y_test,y_pred)
print(classification_report(y_pred,y_test))#ver nro de instancias por clase
crearMatriz_confusion(y_pred,y_test)

print('################## 6. Validar resultados de Modelo ####################')
score = {'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score, average='weighted'),
         'recall': make_scorer(recall_score, average='weighted'),'f1_score': make_scorer(f1_score, average= 'weighted')}

eval_resultado = cross_validate(estimator=clf_rf, X=X_train, y=y_train, cv=skf, scoring=score)
print('Accuracy: ',  (np.mean(eval_resultado['test_accuracy'])*100))
print('Precision: ', (np.mean(eval_resultado['test_precision'])*100))
print('Recall: ', (np.mean(eval_resultado['test_recall'])*100))
print('F1-Score: ', (np.mean(eval_resultado['test_f1_score'])*100))


"""Grafica de entrenamiento del modelo"""
param_range = np.arange(1, (grid_search.best_params_['max_depth']+1),1)
train_scores, test_scores = validation_curve(clf_rf, X_train, y_train, 
                            param_name="max_depth", param_range=param_range,
                                             scoring="accuracy", n_jobs=-1)
"""calcular el promedio y desviacion standar del entrenamiento """
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
"""calcular el promedio y desviacion standar del conjunto de prueba """
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

"""mostrar valores obtenidos del conjunto de entrenamiento y prueba"""
plt.plot(param_range, train_scores_mean - train_scores_std,
     label = "Training Score", color = 'r')
plt.plot(param_range, test_scores_mean - test_scores_std,
   label = "Cross Validation Score", color = 'b')
 
"""crear grafica"""
plt.title("Validation Curve for KNN")
plt.xlabel("n_neighbors ")
plt.ylabel("Score")
plt.tight_layout()
plt.legend(loc = 'best')
#plt.savefig('knn_validationcurve.png')
plt.show()

"""CURVA ROC"""
y_lbtest = label_binarize(y_test,classes=[0, 1, 2, 3])
y_lbpred = label_binarize(y_pred,classes=[0, 1, 2, 3])
fpr = dict()
tpr = dict()
thresh=dict()
roc_auc = dict()
for i in range(4):
    fpr[i], tpr[i], thresh[i] = roc_curve(y_lbtest[:,i], y_lbpred[:,i] )
    roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['red', 'darkorange', 'blue','lime'])
clases = ['Normal','Dos','DDoS', 'Probe']
for i, color, clase in zip(range(4), colors, clases):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='{0} (AUC = {1:0.4f})'
             ''.format(clase, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curve ROC KNN Model ')
plt.legend(loc="lower right")
#plt.savefig('knn_curveroc.png')
plt.show()